In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip data/
!wget http://images.cocodataset.org/zips/val2017.zip data/
!wget http://images.cocodataset.org/zips/test2017.zip data/

In [ ]:
!unzip train2017.zip
!unzip val2017.zip
!unzip test2017.zip

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip

In [ ]:
from pycocotools.coco import COCO

In [ ]:
# annotations/instancesfree
# {dataset}2017.json
path = "annotations/instances_train2017.json"
data = COCO(path)

In [ ]:
import numpy as np
import cv2
from os.path import join

In [ ]:
i = 0
images, masks = [], []
for imgId in data.getImgIds():
    
    img = data.loadImgs(imgId)[0]
    
    valid = False
    anns = data.loadAnns(data.getAnnIds(imgId))

    mask = np.zeros((img["height"], img["width"]), dtype=np.byte)
    for ann in anns:
        
        # category id of person is 1
        if ann["category_id"] == 1:
            seg = data.annToMask(ann)
            mask += seg
            valid = True
    
    # if contains person
    if valid:
        
        file_name = img["file_name"]
        # {dataset}2017
        frame = cv2.imread(join("train2017", file_name))
        # frames/{dataset}
        cv2.imwrite(join("data/frames/train", file_name), frame)
        # masks/{dataset}
        cv2.imwrite(join("data/masks/train", file_name), mask)
        i += 1

In [ ]:
!ls -l data/frames/train | wc -l

In [ ]:
# input image shape
img_shape = (256, 256)
batch_size = 16
n_train = 500 * batch_size
n_val = 100 * batch_size

In [ ]:
import cv2
import glob
from os.path import join, expanduser
import numpy as np

In [ ]:
# this handles all requests for data
class generator:
    
    def __init__(self, frames_dir, masks_dir, img_shape, n_data=None, start=0):
        
        frames_path = glob.glob(join(frames_dir, "*.jpg"))
        self.frames_path = sorted(frames_path, key=lambda path: int(path.split('/')[-1].split('.')[0]))
        
        if masks_dir is None:
            self.masks_path = [""] * len(frames_path)
        else:
            masks_path = glob.glob(join(masks_dir, "*.jpg"))
            self.masks_path = sorted(masks_path, key=lambda path: int(path.split('/')[-1].split('.')[0]))
        
        # use all samples
        if n_data is None:
            n_data = len(frames_path)
        self.n_data = n_data
        
        self.img_shape = img_shape
        # used for shuffling
        self.order = np.arange(n_data) + start
        
    def __getitem__(self, key):
        
        if isinstance(key, slice):
            start, stop, step = key.indices(self.n_data)
            frames, masks = [], []
            
            for i in range(start, stop):
                frame, mask = self.get_data(
                    self.frames_path[self.order[i]], self.masks_path[self.order[i]]
                )
                frames.append(frame)
                masks.append(mask)
                
            return np.array(frames), np.array(masks)
        else:
            return self.get_data(
                self.frames_path[self.order[key]], self.masks_path[self.order[key]]
            )
    
    def __len__(self):
        return self.n_data
    
    def get_data(self, frame_path, mask_path):
        
        frame = cv2.imread(frame_path).astype(np.float32)
        mask = cv2.imread(mask_path)
        if mask is not None:
            mask = mask.astype(np.float32)[:,:,:1]
        
        return frame, mask
    
    def iterator(self, batch_size=None, shuffle=False):
        
        if batch_size is None :
            batch_size = self.n_data
        
        base = 0
        while True:
            
            if shuffle and base == 0:
                np.random.shuffle(self.order)
            
            # yield makes function iterateble
            yield self[base:base + batch_size]

            base += batch_size
            # ensure the iterator runs forever
            if base + batch_size > self.n_data:
                base = 0

In [ ]:
train_frames_path = "data-256/frames/train"
train_masks_path = "data-256/masks/train"
train_generator = generator(
    train_frames_path, 
    train_masks_path, 
    img_shape, 
    n_train
)

val_frames_path = "data-256/frames/val"
val_masks_path = "data-256/masks/val"
val_generator = generator(
    val_frames_path, 
    val_masks_path, 
    img_shape, 
    n_val
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
frame, mask = train_generator[0]
frame = frame.astype(np.int32)
mask = mask.astype(np.int32)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
# cv2 read image in BGR mode
plt.imshow(frame[:,:,::-1])

plt.xticks([])
plt.yticks([])

plt.subplot(1, 2, 2)
plt.imshow(mask[:,:,0])

plt.xticks([])
plt.yticks([])

plt.show()

In [ ]:
import segmentation_models as sm
import keras

model = sm.Unet("resnet34", input_shape=img_shape + (3, ))

optimizer = keras.optimizers.Adam(1e-4)
loss = sm.losses.bce_jaccard_loss
metrics = [sm.metrics.iou_score]

model.compile(optimizer, loss,  metrics)
model.summary()

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("./best-unet.h5", 
        save_weights_only=True, 
        save_best_only=True, 
        mode="min"
    ), 
    keras.callbacks.ReduceLROnPlateau()
]

train_iter = train_generator.iterator(batch_size, True)
val_iter = val_generator.iterator(batch_size, True)

In [ ]:
history = model.fit_generator(
    train_iter, 
    steps_per_epoch=len(train_generator) / batch_size, 
    epochs=30, 
    callbacks=callbacks, 
    validation_data=val_iter, 
    validation_steps=len(val_generator) / batch_size
)

model.save_weights("./last-unet.h5")

In [ ]:
val_loss, val_iou_score = model.evaluate_generator(
    val_iter, 
    steps=n_val / batch_size
)
print(f"val_loss: {val_loss:1.4f} - val_iou_score: {val_iou_score:1.4f}")

In [ ]:
# plot training and validation iou_score values
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['iou_score'])
plt.plot(history.history['val_iou_score'])
plt.title('Model iou_score')
plt.ylabel('iou_score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')

# Plot training and validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
n_test = 1000
n_show = 3

# there is no annotations for test dataset, 
# so we split val dataset into validation and test
test_generator = generator(
    val_frames_path, 
    val_masks_path, 
    img_shape, 
    n_test, 
    n_val
)
test_iter = test_generator.iterator(n_show, True)

In [ ]:
frames, _ = next(test_iter)
frames = frames.astype(np.int32)
res = model.predict(frames)

n_show = 3
plt.figure(figsize=(10, 5 * n_show))

for i in range(n_show):

    frame = frames[i][:,:,::-1]
    
    plt.subplot(n_show, 2, i * 2 + 1)
    plt.imshow(frame)
    
    plt.xticks([])
    plt.yticks([])

    # convert mask into three-chennel image
    mask = np.repeat(res[i], 3, axis=2)
    # set average value as treshold to cut off the foreground
    frame[mask < 0.5] *= 0

    plt.subplot(n_show, 2, i * 2 + 2)
    plt.imshow(frame)
    
    plt.xticks([])
    plt.yticks([])
    
plt.show()